Import the necessary libraries.

In [5]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forg

Define Foursquare credentials.

In [6]:
CLIENT_ID = 'LT0DOUA02BSHZWX2JF14G2NUCEFA1GDSJI22F5RGX4R25ECJ' # your Foursquare ID
CLIENT_SECRET = 'N3O2TTJUQECXXYUJ5UTVPWTJJTB2EOBU2LDJ2Z4IP11A5SVI' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LT0DOUA02BSHZWX2JF14G2NUCEFA1GDSJI22F5RGX4R25ECJ
CLIENT_SECRET:N3O2TTJUQECXXYUJ5UTVPWTJJTB2EOBU2LDJ2Z4IP11A5SVI


First of all, scrape education/school data from the website https://www.elmundo.es/mejores-colegios.html.

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://www.elmundo.es/mejores-colegios.html")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,Puesto,Colegio,Provincia,Tipo,Punt.,Evolución
0,1º,Meres,Asturias,Privado,96,sube
1,2º,Estudio,Madrid,Privado,96,sube
2,3º,Manuel Peleteiro,La Coruña,Privado,96,baja
3,4º,Montserrat,Barcelona,Concertado,96,baja
4,5º,La Salle Ntra. Sra. Maravillas,Madrid,Privado,96,sube


Now, delete all the rows which are schools that are not in the province of Barcelona, and the unnecessary columns.

In [8]:
df = df.rename(columns={"Colegio": "School"})
df = df.rename(columns={"Provincia": "Province"})
df = df.drop("Puesto", axis=1)
df = df.drop("Tipo", axis=1)
df = df.drop("Punt.", axis=1)
df = df.drop("Evolución", axis=1)
df = df[df.Province == 'Barcelona']
df = df.reset_index()
df = df.drop("index", axis=1)

df.head()

,School,Province
0,Montserrat,Barcelona
1,Ágora Sant Cugat,Barcelona
2,La Miranda,Barcelona
3,Virolai,Barcelona
4,La Vall,Barcelona


Now we need to locate the schools and get their latitude and longitude. We use Foursquare to find the coordinates of the schoolsusing the address and we add them to new columns in the above dataframe.

In [9]:
#Montserrat school

address = 'Av. de Vallvidrera, 68, 08017 Barcelona'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
lat_1 = location.latitude
long_1 = location.longitude

#Ágora school
#Foursquare cannot find it so we drop the row
df = df.drop([1], axis=0)


#La Miranda school

address3 = 'Carrer del Canigó, 15, 08960 Sant Just Desvern, Barcelona'
location3 = geolocator.geocode(address3)
lat_3 = location3.latitude
long_3 = location3.longitude

#Virolai school

address4 = 'Carrer de Ceuta, s/n, 08032 Barcelona'
location4 = geolocator.geocode(address4)
lat_4 = location4.latitude
long_4 = location4.longitude

#La Vall school

address5 = 'Carretera de Sabadell, 6, 08193, Barcelona'
location5 = geolocator.geocode(address5)
lat_5 = location5.latitude
long_5 = location5.longitude

#IPSI school

address6 = 'Carrer del Comte Borrell, 243, 08029 Barcelona'
location6 = geolocator.geocode(address6)
lat_6 = location6.latitude
long_6 = location6.longitude

#La salle school

address7 = 'Passeig de la Bonanova, 8, 08022 Barcelona'
location7 = geolocator.geocode(address7)
lat_7 = location7.latitude
long_7 = location7.longitude

#Padre damián school

address8 = 'Av. de Vallvidrera, 10, 08017 Barcelona'
location8 = geolocator.geocode(address8)
lat_8 = location8.latitude
long_8 = location8.longitude

#Put in dataframe

latitude = [lat_1, lat_3, lat_4, lat_5, lat_6, lat_7, lat_8]
longitude = [long_1, long_3, long_4, long_5, long_6, long_7, long_8]

df['Latitude'] = latitude
df['Longitude'] = longitude

df.head()

,School,Province,Latitude,Longitude
0,Montserrat,Barcelona,41.409435,2.110628
2,La Miranda,Barcelona,41.387617,2.083979
3,Virolai,Barcelona,41.418489,2.151749
4,La Vall,Barcelona,41.500750,2.033292
5,IPSI,Barcelona,41.386869,2.149961


Now that we have the school data in order, find out the districts and their coordinates. Use Barcelona districts and neighbourhood data found on the Open Data BCN website, previously downloaded as CSV and added to the notebook.

In [11]:
# The code was removed by Watson Studio for sharing.

,CODI_DISTRICTE,NOM_DISTRICTE,CODI_BARRI,NOM_BARRI
0,1,Ciutat Vella,1,el Raval
1,1,Ciutat Vella,2,el Barri Gòtic
2,1,Ciutat Vella,3,la Barceloneta
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,2,Eixample,5,el Fort Pienc


Merge the districts together and get rid of unnecessary columns.

In [12]:
geo = geo.drop("CODI_DISTRICTE", axis=1)
geo = geo.drop("CODI_BARRI", axis=1)
geo = geo.rename(columns={"NOM_DISTRICTE": "District"})
geo = geo.rename(columns={"NOM_BARRI": "Neighbourhoods"})

geo=geo.groupby(['District'], as_index='false').agg(lambda col: ', '.join(col))

geo = geo.reset_index()
geo.head()

,District,Neighbourhoods
0,Ciutat Vella,"el Raval, el Barri Gòtic, la Barceloneta, Sant..."
1,Eixample,"el Fort Pienc, la Sagrada Família, la Dreta de..."
2,Gràcia,"Vallcarca i els Penitents, el Coll, la Salut, ..."
3,Horta-Guinardó,"el Baix Guinardó, Can Baró, el Guinardó, la Fo..."
4,Les Corts,"les Corts, la Maternitat i Sant Ramon, Pedralbes"


Now we find the coordinates of the districts.

In [14]:

address = 'Ciutat Vella, Barcelona'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
lat_1 = location.latitude
long_1 = location.longitude

address2 = 'Eixample, Barcelona'
location2 = geolocator.geocode(address2)
lat_2 = location2.latitude
long_2 = location2.longitude

address3 = 'Gràcia, Barcelona'
location3 = geolocator.geocode(address3)
lat_3 = location3.latitude
long_3 = location3.longitude

address4 = 'Horta-Guinardó, Barcelona'
location4 = geolocator.geocode(address4)
lat_4 = location4.latitude
long_4 = location4.longitude

address5 = 'Les Corts, Barcelona'
location5 = geolocator.geocode(address5)
lat_5 = location5.latitude
long_5 = location5.longitude

address6 = 'Nou Barris, Barcelona'
location6 = geolocator.geocode(address6)
lat_6 = location6.latitude
long_6 = location6.longitude

address7 = 'Sant Andreu, Barcelona'
location7 = geolocator.geocode(address7)
lat_7 = location7.latitude
long_7 = location7.longitude

address8 = 'Sant Martí, Barcelona'
location8 = geolocator.geocode(address8)
lat_8 = location8.latitude
long_8 = location8.longitude

address9 = 'Sants-Montjuïc, Barcelona'
location9 = geolocator.geocode(address9)
lat_9 = location9.latitude
long_9 = location9.longitude

address10 = 'Sarrià-Sant Gervasi, Barcelona'
location10 = geolocator.geocode(address10)
lat_10 = location10.latitude
long_10 = location10.longitude

#Put in dataframe

latitude = [lat_1, lat_2, lat_3, lat_4, lat_5, lat_6, lat_7, lat_8, lat_9, lat_10]
longitude = [long_1, long_2, long_3, long_4, long_5, long_6, long_7, long_8, long_9, long_10]

geo['Latitude'] = latitude
geo['Longitude'] = longitude

geo = geo.drop("Neighbourhoods", axis=1)

geo.head()

,District,Latitude,Longitude
0,Ciutat Vella,41.374962,2.173265
1,Eixample,41.393394,2.166085
2,Gràcia,41.410171,2.155136
3,Horta-Guinardó,41.428540,2.143597
4,Les Corts,41.388452,2.121718


Now we finally get to the exploration part.

In [15]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Create a map of Barcelona with its districts.

In [16]:
address = 'Barcelona'

geolocator = Nominatim(user_agent="barcelona")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Barcelona are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Barcelona are 41.3828939, 2.1774322.


In [17]:
map_bcn = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district in zip(geo['Latitude'], geo['Longitude'], geo['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bcn)  
    
map_bcn


Explore the districts with Foursquare.

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
bcn_venues = getNearbyVenues(names=geo['District'],
                                   latitudes=geo['Latitude'],
                                   longitudes=geo['Longitude']
                                  )

Ciutat Vella
Eixample
Gràcia
Horta-Guinardó
Les Corts
Nou Barris
Sant Andreu
Sant Martí
Sants-Montjuïc
Sarrià-Sant Gervasi


In [20]:
print(bcn_venues.shape)
bcn_venues.head()

(470, 7)


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ciutat Vella,41.374962,2.173265,The Fish & Chips Shop,41.375965,2.174152,Bistro
1,Ciutat Vella,41.374962,2.173265,macera,41.375589,2.170493,Cocktail Bar
2,Ciutat Vella,41.374962,2.173265,Andante Hotel Roof Pool,41.376565,2.172650,Pool
3,Ciutat Vella,41.374962,2.173265,Andante Hotel Barcelona,41.376556,2.172587,Hotel
4,Ciutat Vella,41.374962,2.173265,Marea Alta,41.376484,2.175106,Seafood Restaurant


In [21]:
print('There are {} uniques categories.'.format(len(bcn_venues['Venue Category'].unique())))


There are 143 uniques categories.


In [22]:
# one hot encoding
onehot = pd.get_dummies(bcn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
onehot['District'] = bcn_venues['District'] 

# move neighbourhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,District,Accessories Store,American Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Building,Burger Joint,Café,Cambodian Restaurant,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cultural Center,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food Stand,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Kids Store,Korean Restaurant,Kosher Restaurant,Light Rail Station,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,National Park,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pub,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Toy / Game Store,Train Station,Tram Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Ciutat Vella,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ciutat Vella,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ciutat Vella,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ciutat Vella,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ciutat Vella,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
bcn_grouped = onehot.groupby('District').mean().reset_index()
bcn_grouped

,District,Accessories Store,American Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Breakfast Spot,Brewery,Building,Burger Joint,Café,Cambodian Restaurant,Castle,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Cultural Center,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food Stand,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Kids Store,Korean Restaurant,Kosher Restaurant,Light Rail Station,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Mountain,Movie Theater,Multiplex,Museum,National Park,Nightclub,Noodle House,Optical Shop,Outdoor Sculpture,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pub,Restaurant,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Sporting Goods Shop,Stadium,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Toy / Game Store,Train Station,Tram Station,Udon Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Ciutat Vella,0.00,0.000000,0.000000,0.011494,0.022989,0.000,0.000000,0.000000,0.000000,0.011494,0.045977,0.00,0.00,0.00,0.011494,0.011494,0.068966,0.0000,0.0000,0.00,0.000000,0.011494,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.011494,0.0000,0.08046,0.000000,0.000000,0.011494,0.000000,0.000,0.0000,0.00,0.0000,0.00,0.000000,0.000000,0.0000,0.011494,0.000000,0.000000,0.0000,0.0000,0.0000,0.011494,0.000000,0.0000,0.000000,0.000000,0.011494,0.011494,0.000000,0.0000,0.00,0.000000,0.011494,0.011494,0.000000,0.0000,0.00,0.00,0.011494,0.103448,0.00,0.011494,0.000000,0.011494,0.011494,0.000000,0.00,0.0000,0.0000,0.011494,0.000000,0.0000,0.0000,0.00,0.000000,0.0000,0.000000,0.034483,0.011494,0.00,0.011494,0.0000,0.011494,0.0000,0.011494,0.000000,0.011494,0.011494,0.0000,0.011494,0.000000,0.000000,0.00,0.022989,0.011494,0.000000,0.0000,0.0000,0.000000,0.011494,0.022989,0.0000,0.057471,0.011494,0.011494,0.011494,0.00,0.00,0.011494,0.0000,0.000000,0.022989,0.0000,0.000,0.011494,0.011494,0.00,0.0,0.00,0.045977,0.0000,0.00,0.000000,0.000000,0.000000,0.045977,0.0000,0.045977,0.0000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000
1,Eixample,0.01,0.000000,0.000000,0.010000,0.000000,0.000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.02,0.02,0.000000,0.010000,0.000000,0.0100,0.0000,0.07,0.000000,0.000000,0.000000,0.0200,0.020000,0.0000,0.000000,0.0000,0.000000,0.0300,0.03000,0.020000,0.000000,0.000000,0.000000,0.010,0.0000,0.01,0.0000,0.01,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0100,0.0000,0.000000,0.000000,0.0300,0.000000,0.010000,0.000000,0.000000,0.000000,0.0100,0.01,0.000000,0.010000,0.000000,0.000000,0.0000,0.00,0.00,0.030000,0.150000,0.02,0.000000,0.010000,0.000000,0.000000,0.010000,0.02,0.0000,0.0000,0.000000,0.000000,0.0000,0.0100,0.01,0.040000,0.0000,0.000000,0.000000,0.000000,0.01,0.020000,0.0000,0.000000,0.0000,0.010000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.020000,0.0000,0.000000,0.000000

In [29]:
num_top_venues = 5

for hood in  bcn_grouped['District']:
    print("----"+hood+"----")
    temp = bcn_grouped[bcn_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ciutat Vella----
           venue  freq
0          Hotel  0.10
1   Cocktail Bar  0.08
2  Boat or Ferry  0.07
3          Plaza  0.06
4            Bar  0.05


----Eixample----
                      venue  freq
0                     Hotel  0.15
1                  Boutique  0.07
2          Tapas Restaurant  0.05
3  Mediterranean Restaurant  0.04
4        Spanish Restaurant  0.04


----Gràcia----
                      venue  freq
0                       Bar  0.12
1  Mediterranean Restaurant  0.06
2             Historic Site  0.05
3                Restaurant  0.04
4                      Park  0.04


----Horta-Guinardó----
            venue  freq
0  Soccer Stadium  0.10
1            Park  0.10
2             Gym  0.10
3    Tennis Court  0.05
4  Farmers Market  0.05


----Les Corts----
                venue  freq
0               Hotel  0.09
1          Restaurant  0.09
2              Bakery  0.06
3                Park  0.06
4  Spanish Restaurant  0.06


----Nou Barris----
                 ve

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['District'] = bcn_grouped['District']

for ind in np.arange(bcn_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bcn_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,District,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ciutat Vella,Hotel,Cocktail Bar,Boat or Ferry,Plaza,Theater,Tapas Restaurant,Spanish Restaurant,Bar,Mexican Restaurant,Seafood Restaurant
1,Eixample,Hotel,Boutique,Tapas Restaurant,Mediterranean Restaurant,Spanish Restaurant,Furniture / Home Store,Hostel,Clothing Store,Cocktail Bar,Seafood Restaurant
2,Gràcia,Bar,Mediterranean Restaurant,Historic Site,Tapas Restaurant,Restaurant,Park,Sandwich Place,Supermarket,Grocery Store,Plaza
3,Horta-Guinardó,Park,Gym,Soccer Stadium,Outdoor Sculpture,Basketball Court,Mediterranean Restaurant,Farmers Market,Plaza,Chinese Restaurant,Soccer Field
4,Les Corts,Hotel,Restaurant,Park,Spanish Restaurant,Bakery,Italian Restaurant,Plaza,Bar,Garden,Tram Station
5,Nou Barris,Fried Chicken Joint,Falafel Restaurant,Metro Station,Gym Pool,Gym,Skate Park,Castle,Food Stand,Food,Flower Shop
6,Sant Andreu,Clothing Store,Burger Joint,Tapas Restaurant,Electronics Store,Sporting Goods Shop,Sandwich Place,Coffee Shop,Spanish Restaurant,Fast Food Restaurant,Restaurant
7,Sant Martí,Pizza Place,Mediterranean Restaurant,Indian Restaurant,Restaurant,Hotel,Park,Bakery,Falafel Restaurant,Brewery,Convenience Store
8,Sants-Montjuïc,Restaurant,History Museum,Metro Station,Clothing Store,Exhibit,Food,Flower Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
9,Sarrià-Sant Gervasi,Light Rail Station,BBQ Joint,Building,Café,General Entertainment,National Park,Farmers Market,Convenience Store,Plaza,Ice Cream Shop


We can see from the above that we will definitely discard Eixample and Ciutat Vella as appropriate districts for settling as a family because their most frequent venue types are hotels, cocktail bars, youth hostels, and many restaurants, with no parks appearing on any of their fequent venues list, implying that they will be noisy during the night. Therefore, we will not consider them as candidates anymore.

Now we need to see where the best schools are, and later we will sort the chosen districts according to the amount of cultural places, parks, and family-friendly fun there is.

The follwoing is a map of the districts and the schools.

In [30]:
map_bcn = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district in zip(geo['Latitude'], geo['Longitude'], geo['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bcn)

for lat, lng, school in zip(df['Latitude'], df['Longitude'], df['School']):
    label = '{}'.format(school)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bcn)
    
map_bcn


We can see that the districts of Sant Martí, Nou Barris, and Sant Andreu are very far away from the best schools in the province, therefore we will also discard them.

We are left with: Gràcia, Horta-Guinardó, Les Corts, Sants-Montjuïc, and Sarrià-Sant Gervasí.

Now, calculate the average of the frequency of occurence of interesting venue types in the narrowed down districts.

In [47]:
new_group = bcn_grouped.loc[[2,3,4,8,9], ['District','Park', 'Playground', 'Garden', 'Museum', 'Movie Theater', 'Theater', 'Pool']]
new_group

,District,Park,Playground,Garden,Museum,Movie Theater,Theater,Pool
2,Gràcia,0.037500,0.0125,0.000000,0.025,0.0,0.0125,0.0
3,Horta-Guinardó,0.100000,0.0000,0.000000,0.000,0.0,0.0000,0.0
4,Les Corts,0.055556,0.0000,0.037037,0.000,0.0,0.0000,0.0
8,Sants-Montjuïc,0.000000,0.0000,0.000000,0.000,0.0,0.0000,0.0
9,Sarrià-Sant Gervasi,0.000000,0.0000,0.000000,0.000,0.0,0.0000,0.0


In [48]:
new_group['Average frequency of family-friendly stuff'] = new_group.mean(numeric_only=True, axis=1)
new_group

,District,Park,Playground,Garden,Museum,Movie Theater,Theater,Pool,Average frequency of family-friendly stuff
2,Gràcia,0.037500,0.0125,0.000000,0.025,0.0,0.0125,0.0,0.012500
3,Horta-Guinardó,0.100000,0.0000,0.000000,0.000,0.0,0.0000,0.0,0.014286
4,Les Corts,0.055556,0.0000,0.037037,0.000,0.0,0.0000,0.0,0.013228
8,Sants-Montjuïc,0.000000,0.0000,0.000000,0.000,0.0,0.0000,0.0,0.000000
9,Sarrià-Sant Gervasi,0.000000,0.0000,0.000000,0.000,0.0,0.0000,0.0,0.000000


From this, we see that Horta-Guinardó should be the optimal district. But, this is biased, because there are a lot of parks in Horta-Guinardó but nothing else (frequency of occurence of the other types of venue is 0 in the above table). Variety is important, therefore we look at the secons highest score. We see that this is Les Corts.

We therefore conclude that the best neighbourhood to raise a family is Les Corts according to our analysis. 